In [1]:
import sys
sys.path.insert(0,"/home/yunye/work/gwaslab/src/")
import gwaslab as gl

In [2]:
mysumstats = gl.Sumstats("t2d_bbj.txt.gz",
             snpid="SNP",
             chrom="CHR",
             pos="POS",
             ea="ALT",
             nea="REF",
             neaf="Frq",
             beta="BETA",
             se="SE",
             p="P",
             direction="Dir",
             build="19",
             n="N", verbose=False)

In [3]:
mysumstats.filter_hapmap3(inplace=True)

2024/02/10 02:00:52 Start to extract HapMap3 SNPs...v3.4.39
2024/02/10 02:00:52  -Current Dataframe shape : 12557761 x 12 ; Memory usage: 1100.88 MB
2024/02/10 02:00:52  -Loading Hapmap3 variants from built-in datasets...
2024/02/10 02:00:53  -Since rsID not in sumstats, CHR:POS( build 19) will be used for matching...
2024/02/10 02:01:14  -Checking if alleles are same...
2024/02/10 02:01:17  -Variants with macthed alleles: 1092430
2024/02/10 02:01:17  -Raw input contains 1092430 Hapmap3 variants based on CHR:POS...
2024/02/10 02:01:17 Finished extracting HapMap3 SNPs.


In [5]:
mysumstats.estimate_h2_by_ldsc(ref_ld_chr = "/home/yunye/tools/ldsc/ldscores/eas_ldscores/", 
                               w_ld_chr = "/home/yunye/tools/ldsc/ldscores/eas_ldscores/")

2024/02/10 02:01:18 Start to extract HapMap3 SNPs...v3.4.39
2024/02/10 02:01:18  -Current Dataframe shape : 1092430 x 13 ; Memory usage: 125.13 MB
2024/02/10 02:01:18  -Loading Hapmap3 variants from built-in datasets...
2024/02/10 02:01:20 Start to run LD score regression...v3.4.39
2024/02/10 02:01:20  -Current Dataframe shape : 1092430 x 13 ; Memory usage: 125.13 MB
2024/02/10 02:01:20  -Run single variate LD score regression:
2024/02/10 02:01:20   -Adopted from LDSC source code: https://github.com/bulik/ldsc
2024/02/10 02:01:20   -Please cite LDSC: Bulik-Sullivan, et al. LD Score Regression Distinguishes Confounding from Polygenicity in Genome-Wide Association Studies. Nature Genetics, 2015.
2024/02/10 02:01:20  -Arguments:
2024/02/10 02:01:20   -ref_ld_chr:/home/yunye/tools/ldsc/ldscores/eas_ldscores/
2024/02/10 02:01:20   -w_ld_chr:/home/yunye/tools/ldsc/ldscores/eas_ldscores/
2024/02/10 02:01:20  -LDSC log:
2024/02/10 02:01:20   -Reading reference panel LD Score from /home/yunye/t

In [8]:
mysumstats.ldsc_h2

,h2_obs,h2_se,Lambda_gc,Mean_chi2,Intercept,Intercept_se,Ratio,Ratio_se
0,0.10394433,0.00650644,1.32982693,1.49125406,1.09147712,0.01056279,0.18621142,0.02150169


In [6]:
mysumstats.estimate_rg_by_ldsc(other_traits=[mysumstats], 
                               rg="t2d,t2d2",
                               ref_ld_chr = "/home/yunye/tools/ldsc/ldscores/eas_ldscores/", 
                               w_ld_chr = "/home/yunye/tools/ldsc/ldscores/eas_ldscores/")

2024/02/10 02:01:27 Start to extract HapMap3 SNPs...v3.4.39
2024/02/10 02:01:27  -Current Dataframe shape : 1092430 x 13 ; Memory usage: 125.13 MB
2024/02/10 02:01:27  -Loading Hapmap3 variants from built-in datasets...
2024/02/10 02:01:28 Start to run LD score regression for genetic correlation...v3.4.39
2024/02/10 02:01:28  -Current Dataframe shape : 1092430 x 13 ; Memory usage: 125.13 MB
2024/02/10 02:01:28  -Run cross-trait LD score regression:
2024/02/10 02:01:28   -Adopted from LDSC source code: https://github.com/bulik/ldsc
2024/02/10 02:01:28   -Please cite LDSC: Bulik-Sullivan, et al. LD Score Regression Distinguishes Confounding from Polygenicity in Genome-Wide Association Studies. Nature Genetics, 2015.
2024/02/10 02:01:28  -Arguments:
2024/02/10 02:01:28   -rg:t2d,t2d2
2024/02/10 02:01:28   -ref_ld_chr:/home/yunye/tools/ldsc/ldscores/eas_ldscores/
2024/02/10 02:01:28   -w_ld_chr:/home/yunye/tools/ldsc/ldscores/eas_ldscores/
2024/02/10 02:01:29 Processing Study_1
2024/02/10 

In [7]:
mysumstats.ldsc_rg

,p1,p2,rg,se,z,p,h2_obs,h2_obs_se,h2_int,h2_int_se,gcov_int,gcov_int_se
0,t2d,t2d2,1.000002,0.000001,919462.741966,0.0,0.102126,0.007921,1.09693,0.019796,1.09693,0.019796
